In [9]:
from math import exp
import pandas as pd
import numpy as np

In [10]:
gamma =1.27437367

In [11]:

data = pd.read_excel("京津冀3省2014-2023使用版数据-2位小数.xlsx")
data = data.iloc[0:, :]  # 这次去取全部数据i=1？
n = data.shape[0]
train_n = n - 2
X0 = data.iloc[:train_n, 1:].to_numpy()
X0

array([[1.83723328, 2.94888886, 5.30014327],
       [1.63016472, 3.39839217, 6.47994269],
       [1.5073275 , 3.29988427, 6.7012894 ],
       [1.59623741, 3.12537946, 7.23209169],
       [1.68514732, 2.95087464, 7.76217765],
       [1.75301896, 2.94091317, 7.18338109],
       [1.93304706, 3.29124123, 8.74641834],
       [2.51918555, 3.3593764 , 9.79154728]])

In [12]:
# 一阶累加生成
X1= np.cumsum(X0, 0)
y1 = X1[:, 0] ** (1 - gamma)

Zy = (y1[1:] + y1[:-1]) / 2
Z2 = (X1[1:, 1] + X1[:-1, 1]) / 2
Z3 = (X1[1:, 2] + X1[:-1, 2]) / 2
# 根据一阶累加生成的第二三列相乘得到序列 pq， 进而得到 Z23
pq = X1[:, 1] * X1[:, 2]
Z23 = (pq[1:] + pq[:-1]) / 2

In [13]:
B = np.array([-Zy, Z2, Z3, Z23]).T
ones_column = np.ones((B.shape[0], 1))
B = (1-gamma)*np.hstack((B, ones_column))
A = y1[1:] - y1[:-1]
    # 共五个值，分别为a, b2, b3, b23, b1

hat_a = np.dot(np.dot(np.linalg.inv(np.dot(B.T, B)), B.T), A)
a, b2, b3, b23, b1 = hat_a
print(a, b2, b3, b23, b1 )

-3.9621653537793944 0.04665101332087834 0.004012482159738678 -0.000273606936845014 -2.8320471189673806


In [14]:
# 这里重新获取下原数据，以实现所有数据的预测
X0 = data.iloc[:, 1:].to_numpy()
X1= np.cumsum(X0, 0)
# 对第一列一阶累加生成求 1-gamma 次幂
y1 = X1[:, 0] ** (1 - gamma)
X0


array([[ 1.83723328,  2.94888886,  5.30014327],
       [ 1.63016472,  3.39839217,  6.47994269],
       [ 1.5073275 ,  3.29988427,  6.7012894 ],
       [ 1.59623741,  3.12537946,  7.23209169],
       [ 1.68514732,  2.95087464,  7.76217765],
       [ 1.75301896,  2.94091317,  7.18338109],
       [ 1.93304706,  3.29124123,  8.74641834],
       [ 2.51918555,  3.3593764 ,  9.79154728],
       [ 2.52289439,  4.41132092, 13.32234957],
       [ 2.56483222,  4.8678233 , 16.10931948]])

In [15]:
  # 根据 y1 求 Zy，再分别根据一阶累加生成的第二三列生成Z2、Z3
Zy = (y1[1:] + y1[:-1]) / 2
Z2 = (X1[1:, 1] + X1[:-1, 1]) / 2
Z3 = (X1[1:, 2] + X1[:-1, 2]) / 2

# 根据一阶累加生成的第二三列相乘得到序列 pq， 进而得到 Z23
pq = X1[:, 1] * X1[:, 2]
Z23 = (pq[1:] + pq[:-1]) / 2

hat_y1 = [y1[0]]
for i in range(n-1):
    hat_y1.append(
           (1 - gamma) / (1 + 0.5 * a * (1 - gamma)) * (b2 * Z2[i] + b3 * Z3[i]) +
           (1 - 0.5 * a * (1 - gamma)) / (1 + 0.5 * a * (1 - gamma)) * hat_y1[-1] +
           (1 - gamma) * b1 / (1 + 0.5 * a * (1 - gamma)) +
           (1 - gamma) / (1 + 0.5 * a * (1 - gamma)) * (b23 * Z23[i])
    )

hat_y1 = np.array(hat_y1)
hat_x1 = np.exp(np.log(hat_y1) / (1 - gamma))
hat_x0 = np.diff(hat_x1)
print(hat_x0)

[1.62518457 1.54452724 1.54351423 1.64432712 1.80283927 2.07033321
 2.35592876 2.6234342  2.54841074]


In [16]:
original_x1 = data.iloc[1:, 1]
APE = np.abs(hat_x0 - original_x1) / original_x1
MAPE=np.mean(APE)
print(APE)
print(MAPE)

1    0.003055
2    0.024679
3    0.033030
4    0.024224
5    0.028420
6    0.071021
7    0.064805
8    0.039851
9    0.006403
Name: 主营业务收入（亿元）, dtype: float64
0.03283185419283626


In [17]:
pd.DataFrame(hat_x0).to_excel('IENGBM_hatx0-case2.xlsx')
pd.DataFrame(APE).to_excel('IENGBM_APE-case2.xlsx')